In [ ]:
from pydub import AudioSegment
import math

In [ ]:
class SplitWavAudioMubin():
    def __init__(self, startFolder, endFolder, filename):
        self.folder = endFolder
        self.filename = filename
        self.filepath = startFolder + '/' + filename
        
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_sec, to_sec, split_filename):
        t1 = from_sec  * 1000
        t2 = to_sec  * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(self.folder + '/' + split_filename, format="wav")
        
    def multiple_split(self, sec_per_split): 
        total_secs = math.ceil(self.get_duration())
        for i in range(0, total_secs, sec_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+sec_per_split, split_fn)
            print(str(i) + ' Done')
            if i == total_secs - sec_per_split:
                print('All splited successfully')

In [ ]:
import os
end_folder = 'Data/segmented_wav'
start_folder = "Data/cleaned_wav"
lsdir = os.listdir(start_folder)
for file in lsdir:
    split_wav = SplitWavAudioMubin(start_folder, end_folder, file)
    split_wav.multiple_split(sec_per_split=10)

In [ ]:
import speech_recognition as sr
base = "C:\\Users\\steel\\Dropbox\\DataScienceProjects\\VoiceSynthesis\\Data\\segmented_wav\\"
from os import path
r = sr.Recognizer()
with sr.AudioFile(base + "700_SUPERCHAT_READING_Thank_you.wav") as source:
    audio = r.record(source)

    

In [ ]:
try:
    print("Sphinx thinks you said " + r.recognize_sphinx(audio))
except sr.UnknownValueError:
    print("Sphinx could not understand audio")
except sr.RequestError as e:
    print("Sphinx error; {0}".format(e))

In [ ]:
AZURE_SPEECH_KEY = "key"  
try:
    print("Microsoft Azure Speech thinks you said " + r.recognize_azure(audio, key=AZURE_SPEECH_KEY))
except sr.UnknownValueError:
    print("Microsoft Azure Speech could not understand audio")
except sr.RequestError as e:
    print("Could not request results from Microsoft Azure Speech service; {0}".format(e))

In [ ]:
import azure.cognitiveservices.speech as speechsdk
filename = base + "700_SUPERCHAT_READING_Thank_you.wav"
filename = base + "780_SUPERCHAT_READING_Thank_you.wav"
filename = base + "1090_SUPERCHAT_READING_Thank_you.wav"
filename = base + "0_SUPERCHAT_READING_Thank_you.wav"
def from_file(file):
    speech_config = speechsdk.SpeechConfig(subscription="SUBSCRIPTION", region='westcentralus')
    audio_input = speechsdk.AudioConfig(filename=file)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)
    
    result = speech_recognizer.recognize_once_async().get()
    return result.text
        


In [ ]:
import os
lsdir = os.listdir(base)
data = []
for file in lsdir:
    text = from_file(base + file)
    if len(text) > 0:
        data.append([base + file, text])
print(data)

In [ ]:
data

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv('speech_rec.csv')

In [ ]:
import pandas as pd
df = pd.read_csv('speech_rec.csv')

In [ ]:
df.head()

In [ ]:
import shutil
import os.path
cnt = 0
for ind, row in df.iterrows():
    new_path = row['0'].split('\\')[-1]
    if os.path.isfile(row['0']):
        shutil.move(row['0'], new_path)
    else:
        cnt += 1
print(cnt)

In [ ]:
df.head()

In [ ]:
for i in df.index:
    new_path =  row['0'].split('\\')[-1]
    print(new_path)
    df['0'][i] = new_path

In [ ]:
cnt = 0
for ind, row in df.iterrows():
    if not os.path.isfile(row['0']):

        cnt += 1
cnt


In [ ]:
df.to_csv('speech_rec_real.csv')

In [ ]:
with open('train.txt', 'w') as file:
    for i, row in df.iterrows():
        print(row['0'].split('\\')[-1]+'|'+row['1'])
        text = (row['0'].split('\\')[-1]+'|'+row['1']).replace(u'\u03c0', 'pi')
        print(text)
        file.write(text + '\n')

In [ ]:
with open('val.txt', 'w') as file:
    for i, row in df.sample(760,axis = 0).iterrows():
        print(row['0'].split('\\')[-1]+'|'+row['1'])
        text = (row['0'].split('\\')[-1]+'|'+row['1']).replace(u'\u03c0', 'pi')
        print(text)
        file.write(text + '\n')

In [ ]:
df.sample(760,axis = 0)